In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout

from tensorflow.keras.layers import GRU, Dropout, Dense

from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import warnings
import os
from datetime import datetime
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'matplotlib'

In [9]:
SEQ_LENGTH =5
RETRAIN_FREQUENCY = 30  # Retrain every 30 days
PERFORMANCE_THRESHOLD = 0.55  # If accuracy drops below this, retrain
MODEL_PATH = 'lstm_stock_model.h5'
SCALER_PATH = 'scaler.npy'
HISTORY_LOG = 'training_history.csv'

# Load your data
df = pd.read_csv('/content/NEPSE_ADX5_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)
df = df[df['date'] > '2013-01-01'].reset_index(drop=True)

# Create target variable
df['target'] = (df['c'].shift(-5) > df['c']).astype(int)
df = df[:-1]
df.columns.tolist()
df.to_csv('orginaldata.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/NEPSE_ADX5_data.csv'

In [ ]:

feature_cols = ['o', 'h', 'l', 'c', 'v', 'psar','PosDI','NegDI','ADX']
# feature_cols = ['accumulated_sentiment']
features_to_use = [col for col in feature_cols if col in df.columns]

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[features_to_use])

# Save scaler for later use
np.save(SCALER_PATH, scaler.data_max_)
scaled_data
df[feature_cols]

,o,h,l,c,v,psar,PosDI,NegDI,ADX
0,532.00,532.00,532.00,532.00,0.000000e+00,521.968604,64.229394,35.770606,49.318599
1,534.00,534.00,534.00,534.00,0.000000e+00,525.394255,67.933096,32.066904,46.628117
2,532.00,532.00,532.00,532.00,0.000000e+00,528.203289,60.148365,39.851635,41.361840
3,529.00,529.00,529.00,529.00,0.000000e+00,541.000000,49.510373,50.489627,33.285323
4,529.00,529.00,529.00,529.00,0.000000e+00,540.760000,49.510373,50.489627,26.824109
...,...,...,...,...,...,...,...,...,...
2916,2662.51,2664.99,2596.04,2604.96,4.629043e+09,2486.920600,12.707651,44.162235,47.926856
2917,2610.28,2610.31,2535.30,2559.59,5.762701e+09,2490.615788,9.508549,53.429926,52.298409
2918,2585.73,2677.09,2575.51,2582.18,5.273221e+09,2494.237072,25.134454,35.788463,45.336265
2919,2583.97,2584.24,2493.16,2503.35,6.707245e+09,2677.090000,19.042366,49.028844,45.079338


In [ ]:
# def create_sequences(data, seq_length=60):
#     print("i have reached sequences")
#     X, y = [],[]
#     for i in range(len(data) - seq_length):
#         print("hello some times \n", i)
#         X.append(data[i:i+seq_length])
#         y.append(data[i+seq_length, -1])
#     return np.array(X), np.array(y)

def create_sequences(X_data, y_data, seq_len=5):
    """
    Create sequences from features and labels separately
    - X_data: feature data (normalized prices/indicators)
    - y_data: label data (target: 0 or 1)
    - seq_len: sequence length (60 days)
    """
    X, y = [], []
    for i in range(len(X_data) - seq_len):
        X.append(X_data[i:i+seq_len])
        y.append(y_data[i+seq_len])  # Take label at position i+seq_len
    return np.array(X), np.array(y)

In [ ]:
def build_model(seq_length, n_features):
    model = Sequential([
        LSTM(64, activation='relu', return_sequences=True,
             input_shape=(seq_length, n_features)),
        Dropout(0.2),
        LSTM(32, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int).flatten()
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy, y_pred

def time_series_train_test_split(X, y, train_size=0.8):
    """
    Split time series data chronologically (not randomly)
    - train_size: proportion of data for training (e.g., 0.8 = 80% train, 20% test)
    """
    split_idx = int(len(X) * train_size)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    return X_train, X_test, y_train, y_test

In [ ]:
def should_retrain(last_retrain_idx, current_idx, retrain_frequency, recent_accuracy):
    """Determine if model should be retrained"""
    days_since_retrain = current_idx - last_retrain_idx

    # Retrain if enough days have passed
    if days_since_retrain >= retrain_frequency:
        return True, "Schedule-based retrain"

    # Retrain if accuracy dropped
    if recent_accuracy is not None and recent_accuracy < PERFORMANCE_THRESHOLD:
        return True, "Performance-based retrain (accuracy dropped)"

    return False, None

In [ ]:
print("="*60)
print("PHASE 1: Initial Pre-training (2010-2021)")
print("="*60)

PHASE 1: Initial Pre-training (2010-2021)


In [ ]:
split_date = pd.Timestamp('2021-01-01')
pre_train_idx = df[df['date'] < split_date].index[-1]
print(pre_train_idx)

# data_with_target = np.column_stack([scaled_data, df['target'].values])
# print(data_with_target)
# X_pre_train, y_pre_train = create_sequences(data_with_target[:pre_train_idx+1], SEQ_LENGTH)
X_pre_train, y_pre_train=create_sequences(scaled_data[:pre_train_idx+1], df['target'].values, SEQ_LENGTH)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_pre_train, y_pre_train, test_size=0.2, random_state=42
# )
X_train, X_val, y_train, y_val = time_series_train_test_split(
    X_pre_train, y_pre_train, train_size=0.8
)
X_train.shape

1805


(1440, 5, 9)

In [ ]:
dfs = pd.DataFrame(data_with_target)
dfs.to_csv('output.csv', index=False)
print(dfs.info())
print(f"Shape: {dfs.shape}")
print(dfs.head())


NameError: name 'data_with_target' is not defined

In [ ]:
model = Sequential([
        LSTM(64, activation='relu', return_sequences=True,
             input_shape=(5, 9)),
        Dropout(0.2),
        LSTM(32, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

# model = Sequential([
#         GRU(64, activation='relu', return_sequences=True,
#              input_shape=(5, 9)),
#         Dropout(0.2),
#         GRU(32, activation='relu', return_sequences=False),
#         Dropout(0.2),
#         Dense(16, activation='relu'),
#         Dense(1, activation='sigmoid')
#     ])

# input_shape=(seq_length,n_feautres)


model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
# early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    epochs=1000,
    batch_size=32,
    validation_data=(X_val, y_val),
    # callbacks=[early_stop],
    verbose=1
)
val_accuracy, _ = evaluate_model(model, X_val, y_val)
print(f"\nPre-training Validation Accuracy: {val_accuracy:.4f}")
model.save(MODEL_PATH)
print("\n" + "="*60)
print("PHASE 2: Fine-tuning & Continuous Learning (2021+)")
print("="*60)

# Start fine-tuning from 2021 onwards
fine_tune_start_idx = pre_train_idx + 1
current_date_idx = fine_tune_start_idx
last_retrain_idx = fine_tune_start_idx
recent_accuracy = None
predictions_log = []

# Simulate daily predictions and periodic retraining
while current_date_idx < len(df) - SEQ_LENGTH:
    current_date = df.iloc[current_date_idx]['date']

    # Make prediction with current model
    if current_date_idx >= SEQ_LENGTH:
        latest_seq = scaled_data[current_date_idx - SEQ_LENGTH:current_date_idx]
        latest_seq_reshaped = latest_seq.reshape(1, SEQ_LENGTH, len(features_to_use))
        pred_prob = model.predict(latest_seq_reshaped, verbose=0)[0][0]
        pred_direction = "UP" if pred_prob > 0.5 else "DOWN"

        predictions_log.append({
            'date': current_date,
            'prediction': pred_direction,
            'confidence': pred_prob if pred_prob > 0.5 else 1 - pred_prob,
            'actual': 'UP' if df.iloc[current_date_idx + 1]['target'] == 1 else 'DOWN'
        })

    # Check if retraining is needed
    should_retrain_flag, reason = should_retrain(
        last_retrain_idx, current_date_idx, RETRAIN_FREQUENCY, recent_accuracy
    )

    if should_retrain_flag and current_date_idx > fine_tune_start_idx + 100:
        print(f"\n{'='*60}")
        print(f"RETRAINING TRIGGERED at {current_date.date()}")
        print(f"Reason: {reason}")
        print(f"{'='*60}")

        # Prepare fine-tuning data from 2021 to current date
        X_ft, y_ft = create_sequences(scaled_data, df['target'].values, SEQ_LENGTH)

        # Split into train/test
        X_ft_train, X_ft_test, y_ft_train, y_ft_test = train_test_split(
            X_ft, y_ft, test_size=0.2, random_state=42
        )

        # Re-compile with lower learning rate for fine-tuning
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.00005),  # Much lower LR
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        # Fine-tune
        history_ft = model.fit(
            X_ft_train, y_ft_train,
            epochs=20,
            batch_size=16,
            validation_data=(X_ft_test, y_ft_test),
            # callbacks=[early_stop],
            verbose=0
        )

        recent_accuracy, _ = evaluate_model(model, X_ft_test, y_ft_test)
        print(f"New Test Accuracy: {recent_accuracy:.4f}")

        model.save(MODEL_PATH)

        # Log retraining
        new_log = pd.DataFrame({
            'date': [datetime.now()],
            'phase': ['fine-tuning'],
            'accuracy': [recent_accuracy],
            'samples_used': [len(X_ft_train)],
            'reason': [reason]
        })
        training_log = pd.concat([training_log, new_log], ignore_index=True)
        training_log.to_csv(HISTORY_LOG, index=False)

        last_retrain_idx = current_date_idx

    current_date_idx += 1

Epoch 1/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.5311 - loss: 0.6928 - val_accuracy: 0.4654 - val_loss: 0.6960
Epoch 2/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5948 - loss: 0.6853 - val_accuracy: 0.4986 - val_loss: 0.7009
Epoch 3/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5891 - loss: 0.6715 - val_accuracy: 0.5568 - val_loss: 0.7013
Epoch 4/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6092 - loss: 0.6676 - val_accuracy: 0.5014 - val_loss: 0.7086
Epoch 5/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6017 - loss: 0.6648 - val_accuracy: 0.4765 - val_loss: 0.7384
Epoch 6/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5976 - loss: 0.6714 - val_accuracy: 0.5263 - val_loss: 0.7137
Epoch 7/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5886 - loss: 0.6798 - val_accuracy: 0.4903 - val_loss: 0.7404
Epoch 8/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5964 - loss: 0.6719 - val_accuracy: 


Pre-training Validation Accuracy: 0.5374

PHASE 2: Fine-tuning & Continuous Learning (2021+)

RETRAINING TRIGGERED at 2021-06-02
Reason: Schedule-based retrain


New Test Accuracy: 0.7175


NameError: name 'training_log' is not defined

In [ ]:
# Initialize training log
training_log = pd.DataFrame({
    'date': [datetime.now()],
    'phase': ['pre-training'],
    'accuracy': [val_accuracy],
    'samples_used': [len(X_train)],
    'reason': ['Initial training on 2010-2021 data']
})
training_log.to_csv(HISTORY_LOG, index=False)


In [ ]:
print("\n" + "="*60)
print("PHASE 2: Fine-tuning & Continuous Learning (2021+)")
print("="*60)

# Start fine-tuning from 2021 onwards
fine_tune_start_idx = pre_train_idx + 1
current_date_idx = fine_tune_start_idx
last_retrain_idx = fine_tune_start_idx
recent_accuracy = None
predictions_log = []

# Simulate daily predictions and periodic retraining
while current_date_idx < len(df) - SEQ_LENGTH:
    current_date = df.iloc[current_date_idx]['date']

    # Make prediction with current model
    if current_date_idx >= SEQ_LENGTH:
        latest_seq = scaled_data[current_date_idx - SEQ_LENGTH:current_date_idx]
        latest_seq_reshaped = latest_seq.reshape(1, SEQ_LENGTH, len(features_to_use))
        pred_prob = model.predict(latest_seq_reshaped, verbose=0)[0][0]
        pred_direction = "UP" if pred_prob > 0.5 else "DOWN"

        predictions_log.append({
            'date': current_date,
            'prediction': pred_direction,
            'confidence': pred_prob if pred_prob > 0.5 else 1 - pred_prob,
            'actual': 'UP' if df.iloc[current_date_idx + 1]['target'] == 1 else 'DOWN'
        })

    # Check if retraining is needed
    should_retrain_flag, reason = should_retrain(
        last_retrain_idx, current_date_idx, RETRAIN_FREQUENCY, recent_accuracy
    )

    if should_retrain_flag and current_date_idx > fine_tune_start_idx + 100:
        print(f"\n{'='*60}")
        print(f"RETRAINING TRIGGERED at {current_date.date()}")
        print(f"Reason: {reason}")
        print(f"{'='*60}")

        # Prepare fine-tuning data from 2021 to current date
        X_ft, y_ft = create_sequences(
            data_with_target[fine_tune_start_idx:current_date_idx+1],
            SEQ_LENGTH
        )

        # Split into train/test
        X_ft_train, X_ft_test, y_ft_train, y_ft_test = train_test_split(
            X_ft, y_ft, test_size=0.2, random_state=42
        )

        # Re-compile with lower learning rate for fine-tuning
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.00005),  # Much lower LR
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        # Fine-tune
        history_ft = model.fit(
            X_ft_train, y_ft_train,
            epochs=20,
            batch_size=16,
            validation_data=(X_ft_test, y_ft_test),
            callbacks=[early_stop],
            verbose=0
        )

        recent_accuracy, _ = evaluate_model(model, X_ft_test, y_ft_test)
        print(f"New Test Accuracy: {recent_accuracy:.4f}")

        model.save(MODEL_PATH)

        # Log retraining
        new_log = pd.DataFrame({
            'date': [datetime.now()],
            'phase': ['fine-tuning'],
            'accuracy': [recent_accuracy],
            'samples_used': [len(X_ft_train)],
            'reason': [reason]
        })
        training_log = pd.concat([training_log, new_log], ignore_index=True)
        training_log.to_csv(HISTORY_LOG, index=False)

        last_retrain_idx = current_date_idx

    current_date_idx += 1

In [ ]:
print("\n" + "="*60)
print("SUMMARY")
print("="*60)

predictions_df = pd.DataFrame(predictions_log)
if len(predictions_df) > 0:
    accuracy = (predictions_df['prediction'] == predictions_df['actual']).sum() / len(predictions_df)
    print(f"Total Predictions Made: {len(predictions_df)}")
    print(f"Overall Prediction Accuracy: {accuracy:.4f}")
    print(f"\nLast 10 Predictions:")
    print(predictions_df.tail(10).to_string())

print(f"\nTraining History saved to: {HISTORY_LOG}")
print(f"Model saved to: {MODEL_PATH}")
print(f"\nRetraining Log:")
print(training_log.to_string())

HELLO NOW LETS TRY OUT SOME SARIMA MODELING!!!!bold text

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import itertools
import warnings
warnings.filterwarnings("ignore")

# ==========================================
# 1️⃣ Load and prepare your data
# ==========================================
df = pd.read_csv("/content/sample_data/NEPSE_ADX_data.csv", parse_dates=['date'])
df = df.sort_values('date')
df.set_index('date', inplace=True)

y = df['c']

# Ensure continuous business-day frequency (optional)
y = y.asfreq('B')
y = y.interpolate()

# ==========================================
# 2️⃣ Define auto SARIMA parameter search
# ==========================================
def auto_sarima(y_train, seasonal_period=5, max_p=2, max_d=1, max_q=2):
    """Finds the best SARIMA (p,d,q)(P,D,Q,s) parameters using AIC."""
    best_aic = np.inf
    best_order = None
    best_seasonal_order = None

    p = d = q = range(0, max_p + 1)
    seasonal_pdq = list(itertools.product(p, d, q))

    for param in itertools.product(p, d, q):
        for seasonal_param in seasonal_pdq:
            seasonal_order = (seasonal_param[0], seasonal_param[1], seasonal_param[2], seasonal_period)
            try:
                model = SARIMAX(
                    y_train,
                    order=param,
                    seasonal_order=seasonal_order,
                    enforce_stationarity=False,
                    enforce_invertibility=False
                )
                results = model.fit(disp=False)
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_order = param
                    best_seasonal_order = seasonal_order
            except:
                continue

    return best_order, best_seasonal_order, best_aic

# ==========================================
# 3️⃣ Rolling validation configuration
# ==========================================
initial_train_end = '2021-12-31'  # train up to end of 2021
forecast_horizon = 30              # forecast next 30 trading days
rolling_steps = 3                  # number of validation splits

mae_scores, rmse_scores = [], []
dates = pd.date_range(start=initial_train_end, end=y.index[-1], freq='Y')
dates = list(dates[:rolling_steps])

for split_date in dates:
    print(f"\n🟢 Rolling split ending on: {split_date.date()}")

    train = y.loc[:split_date]
    test = y.loc[split_date:].iloc[:forecast_horizon]

    # 🔍 Auto-tune SARIMA parameters
    best_order, best_seasonal_order, best_aic = auto_sarima(train)
    print(f"Best params: {best_order} x {best_seasonal_order}, AIC={best_aic:.2f}")

    # Fit best model
    model = SARIMAX(
        train,
        order=best_order,
        seasonal_order=best_seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    results = model.fit(disp=False)

    # Forecast
    forecast_obj = results.get_forecast(steps=len(test))
    forecast = forecast_obj.predicted_mean
    forecast.index = test.index

    # Evaluate
    mae = mean_absolute_error(test, forecast)
    rmse = np.sqrt(mean_squared_error(test, forecast))
    mae_scores.append(mae)
    rmse_scores.append(rmse)

    print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}")

    # Plot each split
    plt.figure(figsize=(10,4))
    plt.plot(train[-100:], label='Train (last 100 days)')
    plt.plot(test, label='Test', color='green')
    plt.plot(forecast, label='Forecast', color='orange')
    plt.title(f'SARIMA Forecast (Split ending {split_date.date()})')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.legend()
    plt.show()

# ==========================================
# 4️⃣ Average validation performance
# ==========================================
print("\n✅ Rolling Validation Summary")
print(f"Average MAE:  {np.mean(mae_scores):.2f}")
print(f"Average RMSE: {np.mean(rmse_scores):.2f}")

# ==========================================
# 5️⃣ Final model on full dataset
# ==========================================
best_order, best_seasonal_order, best_aic = auto_sarima(y)
print(f"\n🏁 Final model params on full data: {best_order} x {best_seasonal_order}")

final_model = SARIMAX(
    y,
    order=best_order,
    seasonal_order=best_seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

future_forecast = final_model.get_forecast(steps=30)
forecast_mean = future_forecast.predicted_mean
conf_int = future_forecast.conf_int()

plt.figure(figsize=(10,5))
plt.plot(y, label='Observed')
plt.plot(forecast_mean, label='Future Forecast', color='orange')
plt.fill_between(conf_int.index,
                 conf_int.iloc[:, 0],
                 conf_int.iloc[:, 1],
                 color='gray', alpha=0.3)
plt.title('Final SARIMA Forecast (Next 30 Trading Days)')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()



🟢 Rolling split ending on: 2021-12-31


NOW LETS TRY TO IMPLEMENT EXPANDING WINDOW LSTM FORECAST

In [ ]:
SEQ_LENGTH = 5
RETRAIN_FREQUENCY = 240  # Retrain every 60 days
PERFORMANCE_THRESHOLD = 0.55  # If accuracy drops below this, retrain
MODEL_PATH = 'lstm_stock_model.h5'
SCALER_PATH = 'scaler.npy'
HISTORY_LOG = 'training_history.csv'
PREDICTIONS_LOG = 'model_predictions.csv'

# Load your data
df = pd.read_csv('/content/NEPSE_ADX5_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)
df = df[df['date'] > '2013-01-01'].reset_index(drop=True)

# Create target variable
df['target'] = (df['c'].shift(-5) > df['c']).astype(int)
df = df[:-1]
df.columns.tolist()
feature_cols = ['o', 'h', 'l', 'c', 'v', 'psar','PosDI','NegDI','ADX']
features_to_use = [col for col in feature_cols if col in df.columns]

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[features_to_use])

# Save scaler for later use
np.save(SCALER_PATH, scaler.data_max_)

def create_sequences(X_data, y_data, seq_len=5):
    """
    Create sequences from features and labels separately
    - X_data: feature data (normalized prices/indicators)
    - y_data: label data (target: 0 or 1)
    - seq_len: sequence length (5 days)
    """
    X, y = [], []
    for i in range(len(X_data) - seq_len):
        X.append(X_data[i:i+seq_len])
        y.append(y_data[i+seq_len])  # Take label at position i+seq_len
    return np.array(X), np.array(y)

def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int).flatten()
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy, y_pred

# Initialize predictions dataframe
predictions_df = pd.DataFrame(columns=['date', 'actual', 'predicted', 'predicted_prob', 'prediction_date', 'model_version'])

# PHASE 1: Initial Training (Pre-2021)
print("="*60)
print("PHASE 1: Initial Model Training (Pre-2021)")
print("="*60)

split_date = pd.Timestamp('2021-01-01')
pre_train_idx = df[df['date'] < split_date].index[-1]
print(f"Training data up to index: {pre_train_idx}")
""
X_pre_train, y_pre_train = create_sequences(
    scaled_data[:pre_train_idx+1],
    df['target'].values[:pre_train_idx+1],
    SEQ_LENGTH
)
X_train, X_val, y_train, y_val = train_test_split(
    X_pre_train, y_pre_train, test_size=0.2, random_state=42, shuffle=True
)

model = Sequential([
    GRU(64, activation='relu', return_sequences=True, input_shape=(SEQ_LENGTH, len(features_to_use))),
    Dropout(0.2),
    GRU(32, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=1000,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

val_accuracy, _ = evaluate_model(model, X_val, y_val)
print(f"\nPre-training Validation Accuracy: {val_accuracy:.4f}")
model.save(MODEL_PATH)

# PHASE 2: Expanding Window Retraining (2021+)
print("\n" + "="*60)
print("PHASE 2: Fine-tuning & Continuous Learning (2021+)")
print("="*60)

# Get all dates after 2021-01-01
post_2021_data = df[df['date'] >= split_date].copy()
post_2021_dates = post_2021_data['date'].unique()
post_2021_dates = np.sort(post_2021_dates)


model_version = 1
current_retrain_date = split_date

# Process in chunks of RETRAIN_FREQUENCY days
for i in range(0, len(post_2021_dates), RETRAIN_FREQUENCY):
    current_retrain_date = post_2021_dates[i]
    print(f"\n--- Retraining with data up to: {current_retrain_date} ---")

    # Find the index for retraining data (ALL REAL HISTORICAL DATA up to current date)
    retrain_end_idx = df[df['date'] <= current_retrain_date].index[-1]

    print(f"Retraining with data from index 0 to {retrain_end_idx} (real historical data only)")

    # Create sequences using ONLY REAL historical data
    X_retrain, y_retrain = create_sequences(
        scaled_data[:retrain_end_idx+1],
        df['target'].values[:retrain_end_idx+1],
        SEQ_LENGTH
    )

    # Split for validation (last 20% of retraining data)
    split_idx = int(len(X_retrain) * 0.8)
    X_retrain_train, X_retrain_val = X_retrain[:split_idx], X_retrain[split_idx:]
    y_retrain_train, y_retrain_val = y_retrain[:split_idx], y_retrain[split_idx:]

    # Fine-tune the model on REAL historical data only (continuing from previous weights)
    print(f"Continuous learning with {len(X_retrain_train)} real historical samples...")

    fine_tune_history = model.fit(
        X_retrain_train, y_retrain_train,
        epochs=600,
        batch_size=32,
        validation_data=(X_retrain_val, y_retrain_val),
        verbose=1
    )

    # Evaluate on validation set
    val_accuracy, _ = evaluate_model(model, X_retrain_val, y_retrain_val)
    print(f"Updated model validation accuracy: {val_accuracy:.4f}")

    # FIXED: Get the next RETRAIN_FREQUENCY trading days by DATE, not by index
    all_future_dates = df[df['date'] > current_retrain_date]['date'].unique()
    prediction_dates = sorted(all_future_dates)[:RETRAIN_FREQUENCY]

    print(f"Making predictions for {len(prediction_dates)} trading days: {prediction_dates[0]} to {prediction_dates[-1]}")

    if len(prediction_dates) > 0:
        pred_dates = []
        predictions = []
        predictions_prob=[]
        actuals = []

        for pred_date in prediction_dates:
            # Get the index of this prediction date in the master CSV
            pred_idx = df[df['date'] == pred_date].index[0]

            # We need at least SEQ_LENGTH previous days of REAL data to make prediction
            if pred_idx >= SEQ_LENGTH:
                # Get the sequence for prediction (using REAL historical features)
                X_pred = scaled_data[pred_idx-SEQ_LENGTH:pred_idx].reshape(
                    1, SEQ_LENGTH, len(features_to_use)
                )

                # Make prediction
                pred_prob = model.predict(X_pred, verbose=0)[0][0]
                pred = 1 if pred_prob > 0.5 else 0

                # Get actual target value from master CSV
                actual_target = df['target'].iloc[pred_idx]

                # Store results
                pred_dates.append(pred_date)
                predictions.append(pred)
                predictions_prob.append(pred_prob)
                actuals.append(actual_target)

                print(f"Date: {pred_date}, Predicted: {pred}, Actual: {actual_target}")

        # Store predictions in dataframe
        if len(pred_dates) > 0:
            batch_predictions = pd.DataFrame({
                'date': pred_dates,
                'actual': actuals,
                'predicted': predictions,
                'predicted_prob': predictions_prob,

                'prediction_date': current_retrain_date,
                'model_version': model_version
            })

            predictions_df = pd.concat([predictions_df, batch_predictions], ignore_index=True)

    # Save updated model
    model_version += 1
    updated_model_path = f"lstm_stock_model_v{model_version}.h5"
    model.save(updated_model_path)
    print(f"Model saved as {updated_model_path}")

    print(f"Completed cycle. Retrained up to: {current_retrain_date}")
    print(f"Made predictions for {len(pred_dates)} trading days")
    print(f"Total predictions so far: {len(predictions_df)}")

# Save all predictions to CSV
predictions_df.to_csv(PREDICTIONS_LOG, index=False)
print(f"\nAll predictions saved to {PREDICTIONS_LOG}")
print(f"Final total predictions: {len(predictions_df)}")

# Calculate overall accuracy
if len(predictions_df) > 0:
    overall_accuracy = accuracy_score(predictions_df['actual'], predictions_df['predicted'])
    print(f"Overall prediction accuracy: {overall_accuracy:.4f}")

# Display final predictions summary
print("\n" + "="*60)
print("PREDICTIONS SUMMARY")
print("="*60)
print(predictions_df.tail(10))

# VERIFY ALL PREDICTION DATES EXIST IN MASTER CSV
print("\n" + "="*60)
print("VALIDATING PREDICTION DATES")
print("="*60)
all_pred_dates = predictions_df['date'].unique()
missing_count = 0

for pred_date in all_pred_dates:
    if pred_date not in df['date'].values:
        print(f"❌ MISSING: {pred_date}")
        missing_count += 1

if missing_count == 0:
    print("✅ SUCCESS: All prediction dates exist in master CSV!")
else:
    print(f"❌ FAILED: {missing_count} prediction dates not found in master CSV")

PHASE 1: Initial Model Training (Pre-2021)
Training data up to index: 1805
Epoch 1/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.5442 - loss: 0.6908 - val_accuracy: 0.5097 - val_loss: 0.6894
Epoch 2/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5597 - loss: 0.6834 - val_accuracy: 0.5817 - val_loss: 0.6917
Epoch 3/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5765 - loss: 0.6802 - val_accuracy: 0.5097 - val_loss: 0.7007
Epoch 4/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5716 - loss: 0.6780 - val_accuracy: 0.4792 - val_loss: 0.7206
Epoch 5/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6170 - loss: 0.6634 - val_accuracy: 0.5208 - val_loss: 0.7117
Epoch 6/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6118 - loss: 0.6597 - val_accuracy: 0.5125 - val_loss: 0.7122
Epoch 7/1000
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5990 - loss: 0.6646 - val_accuracy: 0.5014 - val_loss: 0.7137
Epoch 8/1000
45/45 ━━━━━


Pre-training Validation Accuracy: 0.5097

PHASE 2: Fine-tuning & Continuous Learning (2021+)

--- Retraining with data up to: 2021-01-03T00:00:00.000000000 ---
Retraining with data from index 0 to 1806 (real historical data only)
Continuous learning with 1441 real historical samples...
Epoch 1/600
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9550 - loss: 0.0994 - val_accuracy: 0.5374 - val_loss: 3.5767
Epoch 2/600
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9568 - loss: 0.1050 - val_accuracy: 0.5208 - val_loss: 3.4741
Epoch 3/600
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9737 - loss: 0.0679 - val_accuracy: 0.5291 - val_loss: 3.6016
Epoch 4/600
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9695 - loss: 0.0870 - val_accuracy: 0.5568 - val_loss: 3.5690
Epoch 5/600
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9211 - loss: 0.2237 - val_accuracy: 0.5485 - val_loss: 2.8975
Epoch 6/600
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9542 -

Date: 2021-03-31 00:00:00, Predicted: 0, Actual: 1
Date: 2021-04-01 00:00:00, Predicted: 0, Actual: 1
Model saved as lstm_stock_model_v2.h5
Completed cycle. Retrained up to: 2021-01-03T00:00:00.000000000
Made predictions for 60 trading days
Total predictions so far: 60

--- Retraining with data up to: 2021-04-01T00:00:00.000000000 ---
Retraining with data from index 0 to 1866 (real historical data only)
Continuous learning with 1489 real historical samples...
Epoch 1/600
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9718 - loss: 0.1290 - val_accuracy: 0.5255 - val_loss: 5.9791
Epoch 2/600
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9694 - loss: 0.1291 - val_accuracy: 0.5442 - val_loss: 5.2386
Epoch 3/600
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9617 - loss: 0.1399 - val_accuracy: 0.5308 - val_loss: 5.1495
Epoch 4/600
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9750 - loss: 0.1020 - val_accuracy: 0.5228 - val_loss: 5.5320
Epoch 5/600
47/47 ━━━━━━

Date: 2021-06-29 00:00:00, Predicted: 0, Actual: 0
Model saved as lstm_stock_model_v3.h5
Completed cycle. Retrained up to: 2021-04-01T00:00:00.000000000
Made predictions for 60 trading days
Total predictions so far: 120

--- Retraining with data up to: 2021-06-29T00:00:00.000000000 ---
Retraining with data from index 0 to 1926 (real historical data only)
Continuous learning with 1537 real historical samples...
Epoch 1/600
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9812 - loss: 0.0765 - val_accuracy: 0.5065 - val_loss: 15.8769
Epoch 2/600
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9717 - loss: 0.0797 - val_accuracy: 0.4909 - val_loss: 16.5339
Epoch 3/600
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9761 - loss: 0.0839 - val_accuracy: 0.4987 - val_loss: 15.9700
Epoch 4/600
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9629 - loss: 0.1011 - val_accuracy: 0.5065 - val_loss: 14.6376
Epoch 5/600
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9798

Date: 2021-09-28 00:00:00, Predicted: 0, Actual: 0
Date: 2021-09-29 00:00:00, Predicted: 0, Actual: 1
Model saved as lstm_stock_model_v4.h5
Completed cycle. Retrained up to: 2021-06-29T00:00:00.000000000
Made predictions for 60 trading days
Total predictions so far: 180

--- Retraining with data up to: 2021-09-29T00:00:00.000000000 ---
Retraining with data from index 0 to 1986 (real historical data only)
Continuous learning with 1585 real historical samples...
Epoch 1/600
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9647 - loss: 0.1171 - val_accuracy: 0.5113 - val_loss: 17.4855
Epoch 2/600
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9676 - loss: 0.1208 - val_accuracy: 0.4836 - val_loss: 20.5620
Epoch 3/600
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9577 - loss: 0.1340 - val_accuracy: 0.4962 - val_loss: 20.2901
Epoch 4/600
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9747 - loss: 0.0781 - val_accuracy: 0.4836 - val_loss: 18.4684
Epoch 5/600
50/50 ━

...

Date: 2023-07-05 00:00:00, Predicted: 1, Actual: 1
Date: 2023-07-06 00:00:00, Predicted: 0, Actual: 1
Model saved as lstm_stock_model_v11.h5
Completed cycle. Retrained up to: 2023-04-09T00:00:00.000000000
Made predictions for 60 trading days
Total predictions so far: 600

--- Retraining with data up to: 2023-07-06T00:00:00.000000000 ---
Retraining with data from index 0 to 2406 (real historical data only)
Continuous learning with 1921 real historical samples...
Epoch 1/600
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9640 - loss: 0.2275 - val_accuracy: 0.5468 - val_loss: 7.4649
Epoch 2/600
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9613 - loss: 0.1165 - val_accuracy: 0.5717 - val_loss: 6.8445
Epoch 3/600
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9665 - loss: 0.0891 - val_accuracy: 0.5468 - val_loss: 5.6153
Epoch 4/600
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9650 - loss: 0.0782 - val_accuracy: 0.5696 - val_loss: 6.3190
Epoch 5/600
61/61 ━━━━

Date: 2023-10-04 00:00:00, Predicted: 0, Actual: 0
Date: 2023-10-05 00:00:00, Predicted: 0, Actual: 0
Model saved as lstm_stock_model_v12.h5
Completed cycle. Retrained up to: 2023-07-06T00:00:00.000000000
Made predictions for 60 trading days
Total predictions so far: 660

--- Retraining with data up to: 2023-10-05T00:00:00.000000000 ---
Retraining with data from index 0 to 2466 (real historical data only)
Continuous learning with 1969 real historical samples...
Epoch 1/600
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9651 - loss: 0.1381 - val_accuracy: 0.5517 - val_loss: 2.1088
Epoch 2/600
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9464 - loss: 0.1519 - val_accuracy: 0.5477 - val_loss: 2.4511
Epoch 3/600
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9610 - loss: 0.0973 - val_accuracy: 0.5355 - val_loss: 3.6720
Epoch 4/600
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9790 - loss: 0.0591 - val_accuracy: 0.5355 - val_loss: 4.0452
Epoch 5/600
62/62 ━━━━

Date: 2024-01-21 00:00:00, Predicted: 1, Actual: 0
Model saved as lstm_stock_model_v13.h5
Completed cycle. Retrained up to: 2023-10-05T00:00:00.000000000
Made predictions for 60 trading days
Total predictions so far: 720

--- Retraining with data up to: 2024-01-21T00:00:00.000000000 ---
Retraining with data from index 0 to 2526 (real historical data only)
Continuous learning with 2017 real historical samples...
Epoch 1/600
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9731 - loss: 0.6263 - val_accuracy: 0.4337 - val_loss: 1.5699
Epoch 2/600
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9301 - loss: 0.1650 - val_accuracy: 0.4337 - val_loss: 1.6586
Epoch 3/600
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9684 - loss: 0.0840 - val_accuracy: 0.4455 - val_loss: 2.1828
Epoch 4/600
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9797 - loss: 0.0650 - val_accuracy: 0.4752 - val_loss: 2.1443
Epoch 5/600
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9748 - 

Date: 2024-04-22 00:00:00, Predicted: 1, Actual: 1
Model saved as lstm_stock_model_v14.h5
Completed cycle. Retrained up to: 2024-01-21T00:00:00.000000000
Made predictions for 60 trading days
Total predictions so far: 780

--- Retraining with data up to: 2024-04-22T00:00:00.000000000 ---
Retraining with data from index 0 to 2586 (real historical data only)
Continuous learning with 2065 real historical samples...
Epoch 1/600
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9868 - loss: 0.1007 - val_accuracy: 0.4836 - val_loss: 2.5334
Epoch 2/600
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9690 - loss: 0.0873 - val_accuracy: 0.4836 - val_loss: 2.5927
Epoch 3/600
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9794 - loss: 0.0658 - val_accuracy: 0.4720 - val_loss: 2.8106
Epoch 4/600
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9889 - loss: 0.0376 - val_accuracy: 0.4874 - val_loss: 3.0777
Epoch 5/600
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9794 - 

Date: 2024-07-22 00:00:00, Predicted: 1, Actual: 1
Model saved as lstm_stock_model_v15.h5
Completed cycle. Retrained up to: 2024-04-22T00:00:00.000000000
Made predictions for 60 trading days
Total predictions so far: 840

--- Retraining with data up to: 2024-07-22T00:00:00.000000000 ---
Retraining with data from index 0 to 2646 (real historical data only)
Continuous learning with 2113 real historical samples...
Epoch 1/600
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9563 - loss: 0.3590 - val_accuracy: 0.4612 - val_loss: 2.1842
Epoch 2/600
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9455 - loss: 0.1580 - val_accuracy: 0.4745 - val_loss: 1.7964
Epoch 3/600
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9733 - loss: 0.0830 - val_accuracy: 0.4745 - val_loss: 1.9796
Epoch 4/600
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9723 - loss: 0.0745 - val_accuracy: 0.5104 - val_loss: 2.1545
Epoch 5/600
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9630 - 

In [ ]:
df.index

RangeIndex(start=0, stop=2921, step=1)